# Convert human 1000 genomes phase 3 data from VCF to Zarr

This notebook has example code for converting variant data from the 1000 genomes project phase 3 to Zarr format. 

This notebook uses a single chromosome as an example, however code could be adapted to run all chromosomes.

In [1]:
from pathlib import Path
import sys
import functools
import numpy as np

In [2]:
# Use scikit-allel for the VCF to Zarr conversion.
import allel
allel.__version__

'1.1.10'

In [3]:
# The numcodecs package holds the various compressors that can be used with Zarr.
import numcodecs
from numcodecs import Blosc
numcodecs.__version__

'0.5.5'

In [4]:
import zarr
zarr.__version__

'2.2.0'

## Download and inspect source data

In [5]:
# This is a local directory where we will download VCF files to, and also write Zarr outputs.
data_path = Path('../data/1000genomes/release/20130502')
!mkdir -pv {data_path}

In [6]:
# There is one VCF file per chromosome.
vcf_path_template = str(data_path / 'ALL.chr{chrom}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz')

In [7]:
# Download data for chromosome 22.
!cd {data_path} && wget --no-clobber {vcf_path_template.format(chrom='22')}

File ‘ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz’ already there; not retrieving.



In [8]:
# Inspect file size for interest.
!ls -lh {vcf_path_template.format(chrom='22')}

-rw-r--r-- 1 aliman aliman 205M Jun 21 15:22 ../data/1000genomes/release/20130502/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz


In [9]:
# Inspect which INFO fields are present, for interest.
!zcat {vcf_path_template.format(chrom='22')} | head -n1000 | grep INFO=

##INFO=<ID=CIEND,Number=2,Type=Integer,Description="Confidence interval around END for imprecise variants">
##INFO=<ID=CIPOS,Number=2,Type=Integer,Description="Confidence interval around POS for imprecise variants">
##INFO=<ID=CS,Number=1,Type=String,Description="Source call set.">
##INFO=<ID=END,Number=1,Type=Integer,Description="End coordinate of this variant">
##INFO=<ID=IMPRECISE,Number=0,Type=Flag,Description="Imprecise structural variation">
##INFO=<ID=MC,Number=.,Type=String,Description="Merged calls.">
##INFO=<ID=MEINFO,Number=4,Type=String,Description="Mobile element info of the form NAME,START,END<POLARITY; If there is only 5' OR 3' support for this call, will be NULL NULL for START and END">
##INFO=<ID=MEND,Number=1,Type=Integer,Description="Mitochondrial end coordinate of inserted sequence">
##INFO=<ID=MLEN,Number=1,Type=Integer,Description="Estimated length of mitochondrial insert">
##INFO=<ID=MSTART,Number=1,Type=Integer,Description="Mitochondrial start coordinate of inse

In [10]:
# Inspect which FORMAT fields are present, for interest.
!zcat {vcf_path_template.format(chrom='22')} | head -n1000 | grep FORMAT=

##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">

gzip: stdout: Broken pipe


## Convert VCF to Zarr

In [11]:
# For a lossless conversion from VCF to Zarr, we will need to know the 
# maximum number of alternate alleles found in any variant. This will be used to
# determine the shape of some arrays, like ALT for example.

@functools.lru_cache(maxsize=None)
def find_alt_number(chrom):
    """Scan a VCF to find the maximum number of alleles in any variant."""
    vcf_path = vcf_path_template.format(chrom=chrom)
    callset = allel.read_vcf(vcf_path, fields=['numalt'], log=sys.stdout)
    numalt = callset['variants/numalt']
    return np.max(numalt)

In [12]:
# Demonstrate finding max number of alternate alleles.
find_alt_number('22')

[read_vcf] 65536 rows in 5.26s; chunk in 5.26s (12463 rows/s); 22 :18539397
[read_vcf] 131072 rows in 10.51s; chunk in 5.25s (12486 rows/s); 22 :21016127
[read_vcf] 196608 rows in 16.01s; chunk in 5.50s (11907 rows/s); 22 :23236362
[read_vcf] 262144 rows in 21.13s; chunk in 5.12s (12788 rows/s); 22 :25227844
[read_vcf] 327680 rows in 26.25s; chunk in 5.11s (12815 rows/s); 22 :27285434
[read_vcf] 393216 rows in 31.34s; chunk in 5.09s (12874 rows/s); 22 :29572822
[read_vcf] 458752 rows in 36.55s; chunk in 5.21s (12582 rows/s); 22 :31900536
[read_vcf] 524288 rows in 43.06s; chunk in 6.51s (10067 rows/s); 22 :34069864
[read_vcf] 589824 rows in 48.19s; chunk in 5.13s (12775 rows/s); 22 :36053392
[read_vcf] 655360 rows in 54.19s; chunk in 6.01s (10909 rows/s); 22 :38088395
[read_vcf] 720896 rows in 59.57s; chunk in 5.38s (12181 rows/s); 22 :40216200
[read_vcf] 786432 rows in 64.94s; chunk in 5.37s (12204 rows/s); 22 :42597446
[read_vcf] 851968 rows in 70.29s; chunk in 5.35s (12257 rows/s); 2

8

In [13]:
def build_zarr(zarr_path, chrom, compressor, fields='*'):
    """Run VCF to Zarr conversion for the given chromosome."""
    
    # Determine VCF path for this chromosome.
    vcf_path = vcf_path_template.format(chrom=chrom)
    
    # Zarr can't handle pathlib.Path, ensure string
    zarr_path = str(zarr_path)  
    
    # Determine max number of ALT alleles.
    alt_number = find_alt_number(chrom)
    
    # Run VCF to Zarr converation. For all the options that this function supports, see
    # http://alimanfoo.github.io/2017/06/14/read-vcf.html
    allel.vcf_to_zarr(vcf_path, zarr_path, group=chrom, fields=fields, 
                      alt_number=alt_number, log=sys.stdout, compressor=compressor)


In [14]:
# Choose a compressor - this one is a good allrounder, good compression ratio and reasonable speed.
# Should work well on both local and networked storage.
compressor = Blosc(cname='zstd', clevel=1, shuffle=Blosc.AUTOSHUFFLE)

In [15]:
zarr_path = data_path / 'zarr'

In [16]:
build_zarr(zarr_path, chrom='22', compressor=compressor)

[vcf_to_zarr] 65536 rows in 11.59s; chunk in 11.59s (5653 rows/s); 22 :18539397
[vcf_to_zarr] 131072 rows in 24.01s; chunk in 12.42s (5275 rows/s); 22 :21016127
[vcf_to_zarr] 196608 rows in 36.98s; chunk in 12.97s (5053 rows/s); 22 :23236362
[vcf_to_zarr] 262144 rows in 50.06s; chunk in 13.08s (5009 rows/s); 22 :25227844
[vcf_to_zarr] 327680 rows in 64.26s; chunk in 14.20s (4614 rows/s); 22 :27285434
[vcf_to_zarr] 393216 rows in 77.75s; chunk in 13.49s (4857 rows/s); 22 :29572822
[vcf_to_zarr] 458752 rows in 90.09s; chunk in 12.34s (5310 rows/s); 22 :31900536
[vcf_to_zarr] 524288 rows in 102.63s; chunk in 12.54s (5227 rows/s); 22 :34069864
[vcf_to_zarr] 589824 rows in 115.25s; chunk in 12.61s (5195 rows/s); 22 :36053392
[vcf_to_zarr] 655360 rows in 127.33s; chunk in 12.09s (5421 rows/s); 22 :38088395
[vcf_to_zarr] 720896 rows in 138.22s; chunk in 10.89s (6019 rows/s); 22 :40216200
[vcf_to_zarr] 786432 rows in 149.82s; chunk in 11.60s (5649 rows/s); 22 :42597446
[vcf_to_zarr] 851968 row

## Inspect Zarr output

In [17]:
# Inspect total size of Zarr data.
!du -hs {str(zarr_path)}

110M	../data/1000genomes/release/20130502/zarr


In [18]:
# Inspect size breakdown of Zarr data.
!du -hs {str(zarr_path / '*' / '*' / '*')}

73M	../data/1000genomes/release/20130502/zarr/22/calldata/GT
28K	../data/1000genomes/release/20130502/zarr/22/samples/0
2.0M	../data/1000genomes/release/20130502/zarr/22/variants/AA
1.3M	../data/1000genomes/release/20130502/zarr/22/variants/AC
3.3M	../data/1000genomes/release/20130502/zarr/22/variants/AF
3.3M	../data/1000genomes/release/20130502/zarr/22/variants/AFR_AF
2.4M	../data/1000genomes/release/20130502/zarr/22/variants/ALT
2.7M	../data/1000genomes/release/20130502/zarr/22/variants/AMR_AF
80K	../data/1000genomes/release/20130502/zarr/22/variants/AN
76K	../data/1000genomes/release/20130502/zarr/22/variants/CHROM
80K	../data/1000genomes/release/20130502/zarr/22/variants/CIEND
80K	../data/1000genomes/release/20130502/zarr/22/variants/CIPOS
80K	../data/1000genomes/release/20130502/zarr/22/variants/CS
2.0M	../data/1000genomes/release/20130502/zarr/22/variants/DP
2.4M	../data/1000genomes/release/20130502/zarr/22/variants/EAS_AF
80K	../data/1000genomes/release/20130502/zarr/22/variants

In [19]:
# Open the Zarr data and inspect the hierarchy.
store = zarr.DirectoryStore(str(zarr_path))
callset = zarr.Group(store=store, read_only=True)
callset.tree(expand=True)

/
 └── 22
     ├── calldata
     │   └── GT (1103547, 2504, 2) int8
     ├── samples (2504,) object
     └── variants
         ├── AA (1103547,) object
         ├── AC (1103547, 8) int32
         ├── AF (1103547, 8) float32
         ├── AFR_AF (1103547, 8) float32
         ├── ALT (1103547, 8) object
         ├── AMR_AF (1103547, 8) float32
         ├── AN (1103547,) int32
         ├── CHROM (1103547,) object
         ├── CIEND (1103547, 2) int32
         ├── CIPOS (1103547, 2) int32
         ├── CS (1103547,) object
         ├── DP (1103547,) int32
         ├── EAS_AF (1103547, 8) float32
         ├── END (1103547,) int32
         ├── EUR_AF (1103547, 8) float32
         ├── EX_TARGET (1103547,) bool
         ├── FILTER_PASS (1103547,) bool
         ├── ID (1103547,) object
         ├── IMPRECISE (1103547,) bool
         ├── MC (1103547,) object
         ├── MEINFO (1103547, 4) object
         ├── MEND (1103547,) int32
         ├── MLEN (1103547,) int32
         ├── MSTART (1103547,) int32
         ├── MULTI_ALLELIC (1103547,) bool
         ├── NS (1103547,) int32
         ├── POS (1103547,) int32
         ├── QUAL (1103547,) float32
         ├── REF (1103547,) object
         ├── SAS_AF (1103547, 8) float32
         ├── SVLEN (1103547,) int32
         ├── SVTYPE (1103547,) object
         ├── TSD (1103547,) object
         ├── VT (1103547,) object
         ├── is_snp (1103547,) bool
         ├── numalt (1103547,) int32
         └── svlen (1103547, 8) int32

In [20]:
# Get some diagnostics on the genotype data.
gtz = callset['22/calldata/GT']
gtz.info

Name,/22/calldata/GT
Type,zarr.core.Array
Data type,int8
Shape,"(1103547, 2504, 2)"
Chunk shape,"(65536, 64, 2)"
Order,C
Read-only,True
Compressor,"Blosc(cname='zstd', clevel=1, shuffle=AUTOSHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,5526563376 (5.1G)
No. bytes stored,74640281 (71.2M)


In [22]:
%%time
# Do a quick benchmark of time to compute allele counts over whole chromosome and cohort.

# Wrap Zarr array with scikit-allel class.
gt = allel.GenotypeDaskArray(gtz)

# It helps to know the max number of ALT alleles to expect.
max_allele = callset['22/variants/ALT'].shape[1]

# Run the computation.
ac = gt.count_alleles(max_allele=max_allele).compute()

CPU times: user 30.9 s, sys: 398 ms, total: 31.3 s
Wall time: 4.31 s


In [23]:
# What does an allele counts array look like?
# Rows are variants, columns are alleles, each cell holds the count of observations of an allele for a variant.
ac

<AlleleCountsArray shape=(1103547, 9) dtype=int64>
5007    1    0    0    0    0    0    0    0
4976   32    0    0    0    0    0    0    0
4970   38    0    0    0    0    0    0    0
...
4969   39    0    0    0    0    0    0    0
5007    1    0    0    0    0    0    0    0
4989   19    0    0    0    0    0    0    0